Импорт библиотек

In [1]:
# Ваш код

from numpy import hstack
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
import numpy as np

Подгрузка датасета

In [2]:
# Ваш код

def get_dataset():
    X, y = make_regression(n_samples=10000, n_features=20, n_informative=15, noise=0.1, random_state=7)
    return X, y

X, y = get_dataset()

In [3]:
# Разделение данных на тестовую и остальную выборку
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.5, random_state=1)
# Разделение полученной выборки на обучающую и валидационную
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.33, random_state=1)

print('Train: %s, Val: %s, Test: %s' % (X_train.shape, X_val.shape, X_test.shape))

Train: (3350, 20), Val: (1650, 20), Test: (5000, 20)


Выбор базовых моделей

In [4]:
# Ваш код

def get_models():
    models = list()
    models.append(('LinearRegression', LinearRegression()))
    models.append(('KNeighbors', KNeighborsRegressor()))
    models.append(('DecisionTree', DecisionTreeRegressor(random_state=1)))
    models.append(('SVR', SVR()))
    models.append(('RandomForest', RandomForestRegressor(random_state=1)))
    return models

models = get_models()

Решение с помощью базовых моделей


In [5]:
# Ваш код

#evaluate standalone model
for name, model in models:
  # fit the model on the training dataset
  model.fit(X_train_full, y_train_full)
  # make a prediction on the test dataset
  yhat = model.predict (X_test)
  # evaluate the predictions
  mse = mean_squared_error(y_test, yhat)
  r2 = r2_score(y_test, yhat)
  # report the score
  #print('>%s %s %.3f' % (name, '- Accuracy:', score*100))
  print('>%s %s %.3f %s %.3f' % (name, '- MSE:', mse, '- R2:', r2))

>LinearRegression - MSE: 0.010 - R2: 1.000
>KNeighbors - MSE: 23054.692 - R2: 0.702
>DecisionTree - MSE: 58757.805 - R2: 0.241
>SVR - MSE: 49684.883 - R2: 0.358
>RandomForest - MSE: 20796.999 - R2: 0.731


Решение с помощью блендинга


In [6]:
# Ваш код

# Обучение и прогноз на валидационных данных

def fit_ensemble(models, X_train, X_val, _train, y_val):
  meta_X = list()
  for name, model in models:
    # Обучение моделей на трейн данных
    model.fit(X_train, y_train)
    # Прогноз, формирование мета-признаков
    yhat = model.predict(X_val)
    # Приводим признаки в матричный вид
    yhat = yhat.reshape(len(yhat), 1)
    # Накапливаем в переменной meta_X
    meta_X.append(yhat)
  # Формируем массив для мета-признаков по всем моделям
  meta_X = hstack(meta_X)
  # Cтроим мета-модель
  blender = LinearRegression()
  # Обучаем-мета-модель
  blender.fit(meta_X, y_val)
  return blender

  # Обучение мета-модели на валидационных данных
  blender = fit_ensemble(models, X_train, X_val, y_train, y_val)

In [7]:
# Обучение и прогноз на тестовых данных

def predict_ensemble(models, blender, X_test):
  # make predictions with base models
  meta_X = list()
  for name, model in models:
    # Для каждой базовой модели строим прогноз на тестовых данных
    yhat = model.predict(X_test)
    # Приводим признаки в матричный вид
    yhat = yhat.reshape(len(yhat), 1)
    # Накапливаем в переменной meta X
    meta_X.append(yhat)
  # Формируем массив для мета-признаков по всем моделям
  meta_X = hstack(meta_X)
  # Строим прогноз мета-модель по полученным признакам
  return blender.predict(meta_X)

  # Прогноз мета-модели на тестовых данных
  yhat = predict_ensemble(models, blender, X_test)

In [8]:
# Оценка точности
mse = mean_squared_error(y_test, yhat)
r2 = r2_score(y_test, yhat)
print('>%s %s %.3f %s %.3f' % ('Blending', '- MSE:', mse, '- R2:', r2))

>Blending - MSE: 20796.999 - R2: 0.731


Сравнение результатов

In [ ]:
# Ваш код

>LinearRegression - MSE: 0.010 - R2: 1.000
>KNeighbors - MSE: 23054.692 - R2: 0.702
>DecisionTree - MSE: 58757.805 - R2: 0.241
>SVR - MSE: 49684.883 - R2: 0.358
>RandomForest - MSE: 20796.999 - R2: 0.731

>Blending - MSE: 20796.999 - R2: 0.731